# Insights of Richmond Neighborhood

## Introduction

This project is to take a glimpse at different neighborhoods of Richmond,VA. Richmond is the are capital city of Virginia and is major city in middle Eastern part of the US.
This might be an interest for travelers and Business owners from the different part of the world who want to visit Richmond for various reasons.

## Data

I organized data set of zipcodes and the city neighborhoods with latitude & longtude for the city from multiple online sources.

## Methodology

Learnings from the previous modules and the sources majorly relied on Foursquare API to retrieve all venues of each neighborhoods, then group by each neighborhood and to count how many venues before filter top 10 most common venue types of each neighborhoods.

### Geospatia Data Collected for Richmond, VA. and Converted into the dataframe.

In [41]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_b143b6a64bad4903896a9dd2165690a7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='3PjdP2s9IRaWYKcumNsjiaOkYf4fx6suuAhijpOdY79W',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_b143b6a64bad4903896a9dd2165690a7.get_object(Bucket='capstoneproject-donotdelete-pr-szzmznt8psbliu',Key='Richmond.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

richmond_data = pd.read_csv(body)
richmond_data

,Zipcode,Neighborhood,Latitude,longitude
0,23218,Richmond City,38.71,-85.37
1,23219,Richmond City,38.71,-85.37
2,23220,Richmond City,37.47,-82.75
3,23221,Richmond City,36.76,-83.58
4,23222,Richmond City,37.86,-83.37
5,23223,Henrico County,37.86,-83.37
6,23224,Richmond City,36.84,-84.84
7,23225,Richmond City,36.84,-84.84
8,23226,Richmond City,36.84,-84.84
9,23227,Henrico County,36.84,-84.84


In [6]:
richmond_data['Zipcode'] = richmond_data['Zipcode'].astype(str)

In [43]:
richmond_data.rename(columns={'longitude':'Longitude'},inplace=True)

In [44]:
print(richmond_data.shape)
richmond_data.head()

(39, 4)


,Zipcode,Neighborhood,Latitude,Longitude
0,23218,Richmond City,38.71,-85.37
1,23219,Richmond City,38.71,-85.37
2,23220,Richmond City,37.47,-82.75
3,23221,Richmond City,36.76,-83.58
4,23222,Richmond City,37.86,-83.37


### Import libraries

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Let's get the geographical coordinates of Manhattan.

In [49]:
address = 'Lexington'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Richmond are {}, {}.'.format(latitude, longitude)) 

The geograpical coordinate of Richmond are 38.0464066, -84.4970393.


### Let's Visualize the Neighborhoods in Richmond on Map.

In [51]:
# create map of Richmond using latitude and longitude values
map_richmond = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, neighborhood in zip(richmond_data['Latitude'], richmond_data['Longitude'], richmond_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_richmond)  
    
map_richmond

### Using Foursquare API to explore the neighborhoods

In [13]:
# Working with Foursquare
CLIENT_ID = '1RAQ2BBNPTE2TR0F15FJYYF5B3VA5KWKZZ0HCXIUZTVLCIYZ' # your Foursquare ID
CLIENT_SECRET = '30WXJ3F3QP5DQZE3I4VUWHSUA1GYAUXCTRJA00ZRXEZSUHIM' # your Foursquare Secret
VERSION = '20190722' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 1RAQ2BBNPTE2TR0F15FJYYF5B3VA5KWKZZ0HCXIUZTVLCIYZ
CLIENT_SECRET:30WXJ3F3QP5DQZE3I4VUWHSUA1GYAUXCTRJA00ZRXEZSUHIM


### Now, let's get the top venues that are within a radius of 500 meters.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
venues=getNearbyVenues(df['Neighborhood'], df['Latitude'], df['longitude'])

In [16]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Richmond City,38.71,-85.37,Weight Watchers - Milton School Apartments,38.706930,-85.373603,Weight Loss Center
1,Richmond City,38.71,-85.37,Weight Watchers - Milton School Apartments,38.706930,-85.373603,Weight Loss Center
2,Richmond City,37.47,-82.75,Minnie Park,37.471912,-82.752800,Park
3,Richmond City,36.84,-84.84,Rite Aid,36.840980,-84.839144,Pharmacy
4,Richmond City,36.84,-84.84,Pro Video,36.839894,-84.835502,Entertainment Service


In [17]:
venues.shape

(92, 7)

### Let's check how many venues were returned for each Neighborhood.

In [56]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Chesterfield County,12,12,12,12,12,12
Henrico County,6,6,6,6,6,6
Richmond City,74,74,74,74,74,74


### Analyzing Each Area

In [57]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,ATM,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Entertainment Service,Fast Food Restaurant,Food,Gas Station,Gift Shop,Grocery Store,Gym,Hotel,Insurance Office,Liquor Store,Park,Pharmacy,Pizza Place,Sandwich Place,Steakhouse,Weight Loss Center
0,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Richmond City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Richmond City,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,ATM,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Entertainment Service,Fast Food Restaurant,Food,Gas Station,Gift Shop,Grocery Store,Gym,Hotel,Insurance Office,Liquor Store,Park,Pharmacy,Pizza Place,Sandwich Place,Steakhouse,Weight Loss Center
0,Chesterfield County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000
1,Henrico County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000
2,Richmond City,0.081081,0.054054,0.013514,0.027027,0.013514,0.067568,0.081081,0.027027,0.081081,0.013514,0.013514,0.054054,0.081081,0.013514,0.027027,0.013514,0.013514,0.162162,0.027027,0.027027,0.081081,0.027027


In [59]:
# Define a function for top venus for a neighborhood
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each Neighborhood.

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chesterfield County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
1,Henrico County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
2,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop


## Cluster Neighborhoods
### Run k-means to cluster the neighborhood into 2 clusters.

In [75]:
# set number of clusters
kclusters = 2

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1], dtype=int32)

In [76]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Richmond_merged = richmond_data

# merge Richmond_merged with Richmond data to add latitude/longitude for each neighborhood
Richmond_merged = Richmond_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")

print(Richmond_merged.shape)
Richmond_merged.head()

(39, 15)


,Zipcode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,23218,Richmond City,38.71,-85.37,1,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
1,23219,Richmond City,38.71,-85.37,1,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
2,23220,Richmond City,37.47,-82.75,1,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
3,23221,Richmond City,36.76,-83.58,1,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
4,23222,Richmond City,37.86,-83.37,1,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop


### Finally, let's visualize the resulting clusters

In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Richmond_merged['Latitude'], Richmond_merged['Longitude'], Richmond_merged['Neighborhood'], Richmond_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [66]:
Richmond_merged.loc[Richmond_merged['Cluster Labels'] == 0, Richmond_merged.columns[[1] + list(range(5, Richmond_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Henrico County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
9,Henrico County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
10,Henrico County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
12,Chesterfield County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
13,Chesterfield County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
14,Chesterfield County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
15,Chesterfield County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
19,Henrico County,Pharmacy,Entertainment Service,Steakhouse,Weight Loss Center,Gas Station,American Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store


### Cluster 2

In [67]:
Richmond_merged.loc[Richmond_merged['Cluster Labels'] == 1, Richmond_merged.columns[[1] + list(range(5, Richmond_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
1,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
2,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
3,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
4,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
6,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
7,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
8,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
11,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
16,Richmond City,Pharmacy,ATM,Steakhouse,Entertainment Service,Food,Gym,Discount Store,American Restaurant,Grocery Store,Cosmetics Shop
